In [2]:
#Data Pre-precessing

import numpy as np
import pandas as pd
import csv
from sklearn import preprocessing

def word2num(x,y):
    return [i for i in range(len(y)) if y[i] == x]

def data_precessing(source_datafile,handled_datafile):
    file_data = open(handled_datafile,'w',newline='') #Write dataset

    with open(source_datafile,'r') as data_soure:
        csv_read = csv.reader(data_soure)
        csv_write = csv.writer(file_data)
        for row in csv_read:
            temp = np.array(row)
            
            #Protocol
            protocol_list = ['tcp','udp','icmp']
            if temp[1] in protocol_list:
                temp[1] = word2num(temp[1],protocol_list)[0]

            #Service
            service_list = ['aol','auth','bgp','courier','csnet_ns','ctf','daytime','discard','domain','domain_u',
                    'echo','eco_i','ecr_i','efs','exec','finger','ftp','ftp_data','gopher','harvest','hostnames',
                    'http','http_2784','http_443','http_8001','imap4','IRC','iso_tsap','klogin','kshell','ldap',
                    'link','login','mtp','name','netbios_dgm','netbios_ns','netbios_ssn','netstat','nnsp','nntp',
                    'ntp_u','other','pm_dump','pop_2','pop_3','printer','private','red_i','remote_job','rje','shell',
                    'smtp','sql_net','ssh','sunrpc','supdup','systat','telnet','tftp_u','tim_i','time','urh_i','urp_i',
                    'uucp','uucp_path','vmnet','whois','X11','Z39_50']
            if temp[2] in service_list:
                temp[2] = word2num(temp[2],service_list)[0]
        
            #Flag
            flag_list = ['OTH','REJ','RSTO','RSTOS0','RSTR','S0','S1','S2','S3','SF','SH']
            if temp[3] in flag_list:
                temp[3] = word2num(temp[3],flag_list)[0]

            #State
            if(temp[41] == 'normal'):
                temp[41] = 0
            elif (temp[41] == 'DDoS'):
                temp[41] = 1
            elif (temp[41] == 'UR2'):
                temp[41] = 2
            elif (temp[41] == 'R2L'):
                temp[41] = 3
            else:
                temp[41] = 4
            csv_write.writerow(temp)
        file_data.close()

    return 0
    
def onehot(ori_data, handled_datafile):
    #One-Hot Encoding
    with open('/Users/ziyuanjiang/dstoolbox/set/kddcup_pre') as onehot_file:
        data_matrix = np.loadtxt(onehot_file, delimiter=",")

    with open(ori_data) as onehot_file1:
        data_matrix_source = np.loadtxt(onehot_file1, delimiter=",")

    enc = preprocessing.OneHotEncoder()
    enc.fit(data_matrix[:,[1,2,3]])
    en_matrix = np.zeros((len(data_matrix_source),80))

    for i in range(len(data_matrix_source)):
        en_temp = enc.transform([data_matrix_source[i,[1,2,3]]]).toarray()
        en_matrix[i,:] = en_temp

    
    data_fin = np.hstack((en_matrix,np.delete(data_matrix_source,[1,2,3],axis=1)))
    np.savetxt(handled_datafile,data_fin.astype(int),delimiter=',',fmt='%.0f')
    

In [72]:
source_datafile = '/Users/ziyuanjiang/dstoolbox/set/train'
handled_datafile = '/Users/ziyuanjiang/dstoolbox/set/train_pre'

data_precessing(source_datafile,handled_datafile)
source_datafile = '/Users/ziyuanjiang/dstoolbox/set/kddcup'
handled_datafile = '/Users/ziyuanjiang/dstoolbox/set/kddcup_pre'

data_precessing(source_datafile,handled_datafile)
source_datafile = '/Users/ziyuanjiang/dstoolbox/set/train_pre'
handled_datafile = '/Users/ziyuanjiang/dstoolbox/set/train_onehot'

onehot(source_datafile,handled_datafile)
ource_datafile = '/Users/ziyuanjiang/dstoolbox/set/test'
handled_datafile = '/Users/ziyuanjiang/dstoolbox/set/test_pre'

data_precessing(source_datafile,handled_datafile)
onehot('/Users/ziyuanjiang/dstoolbox/set/test_pre','/Users/ziyuanjiang/dstoolbox/set/test_onehot')
source_datafile = '/Users/ziyuanjiang/dstoolbox/set/labeled'
handled_datafile = '/Users/ziyuanjiang/dstoolbox/set/labeled_pre'

#data_precessing(source_datafile,handled_datafile)
onehot('/Users/ziyuanjiang/dstoolbox/set/labeled_pre','/Users/ziyuanjiang/dstoolbox/set/labeled_onehot')

0

In [3]:

from sklearn import preprocessing
import pandas as pd

file = '/Users/ziyuanjiang/dstoolbox/set/train_onehot'

with open(file) as file_op:
    data_matrix = np.loadtxt(file_op, delimiter=",")

file = '/Users/ziyuanjiang/dstoolbox/set/test_onehot'

with open(file) as file_op:
    test_data = np.loadtxt(file_op, delimiter=",")


In [6]:
from sklearn import preprocessing
import pandas as pd
from sklearn.model_selection import train_test_split
min_max_scaler = preprocessing.MinMaxScaler()
x_minmax = min_max_scaler.fit_transform(data_matrix[:,0:118])
fin_data = np.hstack((x_minmax,data_matrix[:,118:119]))

x_minmax = min_max_scaler.fit_transform(test_data[:,0:118])
fintest_data = np.hstack((x_minmax,test_data[:,118:119]))


train, test = train_test_split(fin_data, test_size=0.3)

In [26]:
with open('/Users/ziyuanjiang/dstoolbox/set/labeled_onehot') as file_op:
    data_matrix = np.loadtxt(file_op, delimiter=",")

x_minmax = min_max_scaler.fit_transform(data_matrix[:,0:118])
fintest_data = np.hstack((x_minmax,data_matrix[:,118:119]))


In [10]:
en_matrix = np.zeros((len(train),5))

for i in range(len(train)):
    en_matrix[i,int(train[i,118:119])] = 1

In [21]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import make_blobs
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import neighbors

X = train[:,0:118]
Y = en_matrix
data = make_blobs(n_samples=311232, n_features=118, centers=5,cluster_std=1.0, random_state=8)  # 此处data为一个元组
#X, Y = data  


clf = neighbors.KNeighborsClassifier()
clf.fit(X, Y)

#result = clf.predict(test_data[:,0:118])



KNeighborsClassifier()

In [23]:
matrix = pd.read_csv('/Users/ziyuanjiang/dstoolbox/set/labeled_test', delimiter=' ', )
del matrix['normal']
matrix.to_csv('/Users/ziyuanjiang/dstoolbox/set/labeled',encoding="utf_8", index=None, header=None)

In [27]:
res_test = clf.predict(fintest_data[:,0:118])

In [17]:
res = np.zeros((len(result),1))
for i in range(len(result)):
    for j in range(5):
        if result[i,j] == 1:
            res[i,0] = j



In [28]:
res = np.zeros((len(res_test),1))
for i in range(len(res_test)):
    for j in range(5):
        if res_test[i,j] == 1:
            res[i,0] = j

from sklearn import metrics
print (sum(res==fintest_data[:,118:119])) 
print(metrics.classification_report(res, fintest_data[:,118:119])) 

[285934]
              precision    recall  f1-score   support

         0.0       1.00      0.72      0.84     83286
         1.0       0.96      1.00      0.98    221933
         2.0       0.00      0.60      0.00        10
         3.0       0.07      0.91      0.12      1006
         4.0       0.75      0.65      0.70      4793

    accuracy                           0.92    311028
   macro avg       0.56      0.78      0.53    311028
weighted avg       0.97      0.92      0.94    311028



In [20]:
from sklearn import metrics
print (sum(res==test_data[:,118:119])) 
print(metrics.classification_report(res, test_data[:,118:119])) 

[41818]
              precision    recall  f1-score   support

         0.0       0.36      0.99      0.53      3611
         1.0       0.97      0.98      0.97     38871
         2.0       0.00      0.00      0.00         0
         3.0       0.43      0.01      0.02      5291
         4.0       0.66      0.16      0.26      1629

    accuracy                           0.85     49402
   macro avg       0.49      0.43      0.36     49402
weighted avg       0.86      0.85      0.82     49402



/Users/ziyuanjiang/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ziyuanjiang/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ziyuanjiang/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
